In [26]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd
import numpy as np

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [96]:
games = session.query(Game).filter(Game.season==2018).all()

In [237]:
G = games_toAdd[500]

In [238]:
pk = G.pk
season = G.season

In [251]:
db.insert_game(G.pk,replace=True)

In [252]:
# query required offensive stats for Rickey's equation 
stmt=f"""
SELECT
    players.fullName,
    players.id,
    p.gamePk,
    p.home_away,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls)+sum(s.intentionalWalks) as BB,
    sum(s.hitByPitch) as HBP,
    sum(s.hits) as hits,
    sum(s.totalBases) as tb,
    sum(s.runs) as runs
FROM 
    game_player_links p
INNER JOIN 
    game_batting_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
AND 
    s.gamePk
IN
    (Select pk from games where season = {season} and type = 'R')
group by 
    players.fullName
having 
    sum(p.gameStatus_isSubstitute)/sum(s.gamesPlayed) < .9
    """

In [253]:
engine = db.db_engine

In [254]:
offense=pd.read_sql(stmt,engine)
offense

,fullName,id,gamePk,home_away,atBats,BB,HBP,hits,tb,runs
0,Addison Russell,608365,529913,away,215.0,17.0,0.0,48.0,65.0,21.0
1,Anthony Rizzo,519203,529913,away,340.0,48.0,12.0,102.0,164.0,42.0
2,Dexter Fowler,451594,529913,home,97.0,13.0,0.0,20.0,36.0,15.0
3,Francisco Pena,506747,529913,home,56.0,6.0,1.0,10.0,11.0,3.0
4,Harrison Bader,664056,529913,home,260.0,24.0,8.0,70.0,111.0,42.0
5,Ian Happ,664023,529913,away,216.0,42.0,2.0,52.0,83.0,32.0
6,Jason Heyward,518792,529913,away,256.0,24.0,0.0,69.0,97.0,38.0
7,Javier Baez,595879,529913,away,360.0,21.0,2.0,111.0,210.0,62.0
8,Jedd Gyorko,576397,529913,home,217.0,33.0,2.0,59.0,97.0,39.0
9,Jose Martinez,500874,529913,home,301.0,23.0,0.0,87.0,121.0,32.0


In [255]:
# query required defensive stats for Rickey's equation 
stmt=f"""
SELECT 
    p.home_away,
    players.fullName,
    sum(s.hits) as hits,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls) as walks,
    sum(s.hitBatsmen) as HBP,
    sum(s.earnedRuns) as er,
    sum(s.inningsPitched)/sum(s.gamesPitched) > 3 
        and
        sum(s.gamesStarted)/sum(s.gamesPitched) > .9
        as starter
FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
AND 
    g.season = {season} and g.type = 'R' and g.dateTime < "{date}"
group by 
    players.fullName
    """

In [256]:
defense = pd.read_sql(stmt,engine)
defense

,home_away,fullName,hits,atBats,walks,HBP,er,starter


once more but in aggregate by teams

In [201]:
date = dt.strftime(G.dateTime,"%Y-%m-%d")
date

'2018-04-29'

In [218]:
#offense
stmt=f"""
SELECT
    p.gamePk,
    p.home_away,
    (
        sum(s.hits) +
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch) 
    ) /
    (   sum(s.atBats) + 
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch)
    ) as offense,
    
    (sum(s.totalBases) - sum(s.hits)) / sum(s.atBats) as power,
    sum(s.runs) / 
    (
        sum(s.hits) +
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch) 
    ) as clutch
FROM 
    game_player_links p
INNER JOIN 
    game_batting_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g on g.pk = s.gamePk
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
AND 
    g.season = {season} and g.type = 'R' and s.gameDateTime < "{date}"
group by 
    p.home_away
    """

In [219]:
off = pd.read_sql(stmt,engine)
off.set_index('home_away',inplace=True)
off

,gamePk,offense,power,clutch
home_away,,,,


In [220]:
# for relievers
stmt=f"""
SELECT 
    p.home_away,
    sum(s.hits) / sum(s.atBats) as r_oppBa,
    ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
    /
    ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppBb,
    sum(s.earnedRuns) 
    /
    ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppEr

FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
    AND 
    g.season = 2019 and g.type = 'R' and g.dateTime < "2019-06-11"
group by 
    p.home_away
having
    sum(s.inningsPitched)/sum(s.gamesPitched) < 3 
    AND
    sum(s.gamesStarted)/sum(s.gamesPitched) < .9
    """

In [114]:
def_relief = pd.read_sql(stmt,engine)
def_relief.set_index('home_away',inplace=True)
def_relief

,r_oppBa,r_oppBb,r_oppEr
home_away,,,
away,0.262295,0.083333,0.379747
home,0.239362,0.079935,0.350543


In [302]:
# for starter
stmt=f"""
SELECT 
    p.home_away,
    sum(s.hits) / sum(s.atBats) as s_oppBa,
    ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
    /
    ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppBb,
    sum(s.earnedRuns) 
    /
    ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppEr

FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk}
    AND 
    g.season = 2019 and g.type = 'R' and g.dateTime < "2019-06-11"
    AND
    players.id = g.home_probablePitcher or players.id = away_probablePitcher
group by 
    p.home_away
    """

In [303]:
starters = pd.read_sql(stmt,engine)
starters.set_index('home_away',inplace=True)
starters

,s_oppBa,s_oppBb,s_oppEr
home_away,,,
away,0.258778,0.089064,0.368402
home,0.258918,0.089008,0.368209


In [304]:
df = pd.concat([off,def_relief,starters],sort=False,axis=1)

In [305]:
df

,gamePk,offense,power,clutch,r_oppBa,r_oppBb,r_oppEr,s_oppBa,s_oppBb,s_oppEr
home_away,,,,,,,,,,
away,567491,0.344774,0.192173,0.366516,0.250000,0.087844,0.353933,0.258778,0.089064,0.368402
home,567491,0.339329,0.192725,0.405444,0.234818,0.090000,0.336222,0.258918,0.089008,0.368209


In [306]:
records = df.to_dict(orient='index')

In [307]:
d = {}
for team in ['home','away']:
    keys = records[team].keys()
    for k in keys:
        if k != 'gamePk':
            d[f"{team}_{k}"] = records[team][k]
        d['gamePk']=records[team]['gamePk']

In [308]:
d

{'gamePk': 567491,
 'home_offense': 0.33932912007778315,
 'home_power': 0.19272529858849077,
 'home_clutch': 0.4054441260744986,
 'home_r_oppBa': 0.23481781376518218,
 'home_r_oppBb': 0.09,
 'home_r_oppEr': 0.3362218370883882,
 'home_s_oppBa': 0.2589178316117167,
 'home_s_oppBb': 0.08900760149141955,
 'home_s_oppEr': 0.36820866398613367,
 'away_offense': 0.34477379095163807,
 'away_power': 0.19217289719626168,
 'away_clutch': 0.3665158371040724,
 'away_r_oppBa': 0.25,
 'away_r_oppBb': 0.0878438331854481,
 'away_r_oppEr': 0.3539325842696629,
 'away_s_oppBa': 0.2587778647469107,
 'away_s_oppBb': 0.08906400106108556,
 'away_s_oppEr': 0.3684018896397273}

In [55]:
game_record = session.query(Game).first()

In [56]:
game_record.type

'R'

In [6]:
dt.strftime(game_record.dateTime,"%Y-%m-%d")

'2019-06-11'

In [7]:
game_record.season

2019

In [86]:
def get_game_features(game_record):
    date = dt.strftime(game_record.dateTime,"%Y-%m-%d")
    pk = game_record.pk
    season = game_record.season
    
    #offense
    stmt=f"""
    SELECT
        p.gamePk,
        p.home_away,
        (
            sum(s.hits) +
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch) 
        ) /
        (   sum(s.atBats) + 
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch)
        ) as offense,

        (sum(s.totalBases) - sum(s.hits)) / sum(s.atBats) as power,
        sum(s.runs) / 
        (
            sum(s.hits) +
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch) 
        ) as clutch
    FROM 
        game_player_links p
    INNER JOIN 
        game_batting_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    WHERE
        p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
    AND 
        s.gamePk
    IN
        (Select pk from games where season = {season} and type = 'R' and dateTime < "{date}")
    group by 
        p.home_away
    having 
        sum(p.gameStatus_isSubstitute)/sum(s.gamesPlayed) < .9
        """
    off = pd.read_sql(stmt,engine)
    off.set_index('home_away',inplace=True)
    
    # for relievers
    stmt=f"""
    SELECT 
        p.home_away,
        sum(s.hits) / sum(s.atBats) as r_oppBa,
        ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
        /
        ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppBb,
        sum(s.earnedRuns) 
        /
        ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppEr

    FROM 
        game_player_links p
    INNER JOIN 
        game_pitching_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    INNER JOIN 
        games g ON g.pk=s.gamePk
    WHERE
        p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
        AND 
        g.season = {season} and g.type = 'R' and g.dateTime < "{date}"
    group by 
        p.home_away
    having
        sum(s.inningsPitched)/sum(s.gamesPitched) < 3 
        AND
        sum(s.gamesStarted)/sum(s.gamesPitched) < .9
        """
    relief = pd.read_sql(stmt,engine)
    relief.set_index('home_away',inplace=True)
    
    # for starter
    stmt=f"""
    SELECT 
        p.home_away,
        sum(s.hits) / sum(s.atBats) as s_oppBa,
        ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
        /
        ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppBb,
        sum(s.earnedRuns) 
        /
        ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppEr

    FROM 
        game_player_links p
    INNER JOIN 
        game_pitching_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    INNER JOIN 
        games g ON g.pk=s.gamePk
    WHERE
        p.gamePk = {pk}
        AND 
        g.season = {season} and g.type = 'R' and g.dateTime < "{date}"
        AND
        players.id = g.home_probablePitcher or players.id = away_probablePitcher
    group by 
        p.home_away
        """
    starters = pd.read_sql(stmt,engine)
    starters.set_index('home_away',inplace=True)
    
    df = pd.concat([off,relief,starters],sort=False,axis=1)
    records = df.to_dict(orient='index')
    
    d = {}
    for team in ['home','away']:
        keys = records[team].keys()
        for k in keys:
            if k != 'gamePk':
                d[f"{team}_{k}"] = records[team][k]
            d['gamePk']=records[team]['gamePk']
    return d

In [115]:
get_game_features(G)

{'gamePk': 531390,
 'home_offense': 0.30900109769484085,
 'home_power': 0.15398442180946675,
 'home_clutch': 0.33570159857904086,
 'home_r_oppBa': 0.24955595026642985,
 'home_r_oppBb': 0.0889967637540453,
 'home_r_oppEr': 0.3375959079283887,
 'home_s_oppBa': 0.2577693919285891,
 'home_s_oppBb': 0.0894837633206651,
 'home_s_oppEr': 0.3609066114857128,
 'away_offense': 0.31942575145805296,
 'away_power': 0.1425073457394711,
 'away_clutch': 0.33146067415730335,
 'away_r_oppBa': 0.25412293853073464,
 'away_r_oppBb': 0.07361111111111111,
 'away_r_oppEr': 0.34831460674157305,
 'away_s_oppBa': 0.25786274245995544,
 'away_s_oppBb': 0.08946337153734955,
 'away_s_oppEr': 0.3611727458033555}

In [57]:
april_1 = dt(2018,4,1)

In [49]:
from sqlalchemy import and_

In [76]:
# add game type filter 
games = session.query(Game).filter(Game.type=='R').all()

In [344]:
data = [get_game_features(game) for game in games]

In [5]:
data = pd.DataFrame.from_records(data)

In [6]:
data.to_csv('dataset.csv',index=False)

In [11]:
df = pd.read_csv('dataset.csv')
df.head()

,gamePk,home_offense,home_power,home_clutch,home_r_oppBa,home_r_oppBb,home_r_oppEr,home_s_oppBa,home_s_oppBb,home_s_oppEr,away_offense,away_power,away_clutch,away_r_oppBa,away_r_oppBb,away_r_oppEr,away_s_oppBa,away_s_oppBb,away_s_oppEr
0,566180,0.363636,0.079365,0.285714,0.163934,0.075758,0.333333,0.258922,0.089009,0.368218,0.315068,0.250000,0.391304,0.253968,0.112676,0.375000,0.258785,0.089063,0.368410
1,565316,0.220000,0.055556,0.227273,0.230769,0.099010,0.354839,0.258922,0.089009,0.368218,0.321739,0.155340,0.405405,0.218750,0.102804,0.468750,0.258785,0.089063,0.368410
2,567459,0.412371,0.187500,0.325000,0.240385,0.079646,0.294118,0.258922,0.089008,0.368218,0.254902,0.074074,0.153846,0.176923,0.084507,0.314286,0.258785,0.089063,0.368410
3,565223,0.238095,0.172414,0.333333,0.223881,0.182927,0.200000,0.258921,0.089010,0.368217,0.335766,0.235294,0.369565,0.237410,0.085526,0.413043,0.258785,0.089064,0.368412
4,566868,0.246479,0.092308,0.342857,0.155556,0.111842,0.157895,0.258919,0.089009,0.368213,0.300885,0.125000,0.411765,0.277228,0.192000,0.288462,0.258785,0.089063,0.368410


In [14]:
def get_homeTeam_win(gamePk):
    gamePk = str(int(gamePk))
    stmt = f"""
    SELECT 
        sum(BS.runs) as runs,
        L.home_away,
        L.gamePk
    FROM 
        game_batting_stats BS
    INNER JOIN 
        game_player_links L
        ON L.person_id = BS.person_id
    WHERE
        L.gamePk = {gamePk}
        and BS.gamePk = {gamePk}
    GROUP BY 
        L.home_away
    ORDER BY 
        runs desc
    LIMIT
        1
    """
    d = {}
    with db.db_engine.connect() as connection:
        result = connection.execute(stmt).fetchone()
        if result==None:
            d['homeTeam_win']= np.nan
        elif result['home_away']=='home':
            d['homeTeam_win']=1
        else:
            d['homeTeam_win']=0
    return d['homeTeam_win']
    

df['homeTeam_win'] = df.apply(lambda x: get_homeTeam_win(x['gamePk']),axis=1)

df.dropna(inplace=True)

In [16]:
df.to_csv('dataset.csv',index=False)

In [62]:
player = session.query(Player).filter(Player.primaryPosition_name=='Pitcher').first()

In [65]:
player_id = player.id

In [80]:
def chunk(n,list_to_chunk):
    """
    takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
    be equal to n. 
    """
    return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

In [82]:
import csv 

reader = csv.DictReader(open("dataset.csv"))
already_added = [row for row in reader] 
already_addedPks = [row['gamePk'] for row in already_added]
keys = already_added[0].keys()
games_toAdd = [ g for g in games if str(g.pk) not in already_addedPks ]

In [78]:
len(games_toAdd)

4920

In [81]:
count=1
chunks = chunk(100,games_toAdd)

In [90]:
engine = db.db_engine

In [180]:
games_toAdd[0]

<Game(pk='529870',id='2018/05/02/tormlb-minmlb-1')>

In [182]:
db.insert_game(games_toAdd[400].pk,replace=True)

In [183]:
get_game_features(games_toAdd[400])

{'gamePk': nan,
 'home_offense': nan,
 'home_power': nan,
 'home_clutch': nan,
 'home_r_oppBa': nan,
 'home_r_oppBb': nan,
 'home_r_oppEr': nan,
 'home_s_oppBa': 0.25776768968319663,
 'home_s_oppBb': 0.08949562397087436,
 'home_s_oppEr': 0.3608608979091441,
 'away_offense': nan,
 'away_power': nan,
 'away_clutch': nan,
 'away_r_oppBa': nan,
 'away_r_oppBb': nan,
 'away_r_oppEr': nan,
 'away_s_oppBa': 0.2578607635674775,
 'away_s_oppBb': 0.08947906988442135,
 'away_s_oppEr': 0.36113219248883666}

In [ ]:
for chunk in chunks:
    